In [1]:

from sklearn.datasets import load_diabetes # Carrega o dataset
diabetes = load_diabetes()
diabetes.keys()

dict_keys(['data', 'target', 'frame', 'DESCR', 'feature_names', 'data_filename', 'target_filename'])

In [2]:
X, y = diabetes.data, diabetes.target
X.shape, y.shape # (442, 10): 442 amostras/exemplos, com 10 características possíveis. (442,): E somente um rótulo/resultado para cada exemplo

((442, 10), (442,))

In [3]:
from sklearn.metrics import mean_squared_error, make_scorer
from sklearn.model_selection import cross_validate # Usar o cross_validate para encontrar os scores

# Não usa o KNeighborsClassified, pois a base diabetes, é uma base de regressão. Então tem que ser o KNeighborsRegressor
from sklearn.neighbors import KNeighborsRegressor 


import numpy as np

modelo = KNeighborsRegressor() # Dentro do parêntesis pode ser colocado manualmente um número, que significa a quantidade de vizinhos. O default é 5.
# Essa quantidade de vizinhos sendo alterada, é considerada ruim, pois você está introduzindo um viés no resultado. Como se vc tivesse induzindo um resultado.
# Você poderia, por exemplo, ficar trocando os valores para obter um resultado muito melhor do que quando colocar o algorítimo em produçaõ.


scores = cross_validate(modelo, X, y, scoring=make_scorer(mean_squared_error, squared=False))

print(scores['test_score']) # Imprimindo somente o test_score (retorno de cross_validate?)
sempad = np.mean(scores['test_score']) 
print(f"Sem padronização: {sempad}") # imprime A média dos scores sem padronização

[56.2778101  63.42066441 59.63695849 59.59086644 61.70551029]
Sem padronização: 60.126361947859564


In [4]:
from sklearn.model_selection import GridSearchCV

parametros = {'n_neighbors': [3,5,7]}

modelo = GridSearchCV(KNeighborsRegressor(), parametros, scoring=make_scorer(mean_squared_error, greater_is_better=False, squared=False))
modelo.fit(X, y)

GridSearchCV(estimator=KNeighborsRegressor(),
             param_grid={'n_neighbors': [3, 5, 7]},
             scoring=make_scorer(mean_squared_error, greater_is_better=False, squared=False))

In [5]:
modelo.cv_results_ # Será exibido todos os scores das 5 folds (split0 `a split4) que foram testadas, os scores de cada e a médias (mean_test_score) dos scores

{'mean_fit_time': array([0.00139751, 0.00117583, 0.00120153]),
 'std_fit_time': array([0.00048706, 0.00040912, 0.00040357]),
 'mean_score_time': array([0.00200229, 0.00202389, 0.00199919]),
 'std_score_time': array([4.79073051e-06, 4.15700826e-05, 1.09231953e-03]),
 'param_n_neighbors': masked_array(data=[3, 5, 7],
              mask=[False, False, False],
        fill_value='?',
             dtype=object),
 'params': [{'n_neighbors': 3}, {'n_neighbors': 5}, {'n_neighbors': 7}],
 'split0_test_score': array([-58.07341774, -56.2778101 , -56.99180879]),
 'split1_test_score': array([-67.43002106, -63.42066441, -62.93518794]),
 'split2_test_score': array([-62.28861222, -59.63695849, -59.57483095]),
 'split3_test_score': array([-62.02315061, -59.59086644, -55.63774296]),
 'split4_test_score': array([-67.25015489, -61.70551029, -60.43146504]),
 'mean_test_score': array([-63.41307131, -60.12636195, -59.11420714]),
 'std_test_score': array([3.53743647, 2.3951518 , 2.5743976 ]),
 'rank_test_scor

In [6]:
modelo.best_estimator_ # Será exibido o mnelhor estimador que houve no processamento anterior

KNeighborsRegressor(n_neighbors=7)

In [7]:
# Fazer uma avaliação utilizando a mesma base está errado, o correto seria dividir entre treino e teste. Mas assim mesmo ele fez
from sklearn.metrics import mean_squared_error
from math import sqrt

ypred = modelo.predict(X) # Calcula o ypred utilizando predição
mse = mean_squared_error(y, ypred) # calcula o erro (MSE)
sqrt(mse) # Tira a raiz quadrada do erro

51.15149901888715

In [8]:
#Fazendo de uma forma mais justa, mas ainda SEM PADRONIZAÇÃO
#### Copiando do cross_validate anterior
scores = cross_validate(modelo, X, y, scoring=make_scorer(mean_squared_error, squared=False), return_estimator=True)
print(scores['test_score'])
sempad = np.mean(scores['test_score'])
print(f"Sem padronização: {sempad}") # O resultado saiu de 51 para 59

[56.99180879 62.93518794 59.57483095 55.63774296 60.43146504]
Sem padronização: 59.114207135103996


In [9]:
# A ideia aqui seria juntar o GridSearchCV com o pipeline
scores

{'fit_time': array([0.04900002, 0.04599977, 0.04300094, 0.04299378, 0.0439992 ]),
 'score_time': array([0.00199938, 0.00200129, 0.00200534, 0.0019989 , 0.00200224]),
 'estimator': (GridSearchCV(estimator=KNeighborsRegressor(),
               param_grid={'n_neighbors': [3, 5, 7]},
               scoring=make_scorer(mean_squared_error, greater_is_better=False, squared=False)),
  GridSearchCV(estimator=KNeighborsRegressor(),
               param_grid={'n_neighbors': [3, 5, 7]},
               scoring=make_scorer(mean_squared_error, greater_is_better=False, squared=False)),
  GridSearchCV(estimator=KNeighborsRegressor(),
               param_grid={'n_neighbors': [3, 5, 7]},
               scoring=make_scorer(mean_squared_error, greater_is_better=False, squared=False)),
  GridSearchCV(estimator=KNeighborsRegressor(),
               param_grid={'n_neighbors': [3, 5, 7]},
               scoring=make_scorer(mean_squared_error, greater_is_better=False, squared=False)),
  GridSearchCV(estimator=

In [10]:
for estimator in scores['estimator']:
    print(estimator.best_estimator_)

KNeighborsRegressor(n_neighbors=7)
KNeighborsRegressor(n_neighbors=7)
KNeighborsRegressor(n_neighbors=7)
KNeighborsRegressor(n_neighbors=7)
KNeighborsRegressor(n_neighbors=7)


In [11]:
from sklearn.preprocessing import StandardScaler # Importando o Scaler
from sklearn.pipeline import Pipeline #Importando o Pipeline

# Há duas formas de fazer:
# A primeira delas é ajustar apenas o estimador no gridsearch e colocar este gridsearch dentro do pipeline. Quando só precisa ajustas os parâmetros do estimador, isso funcionaria muito bem e é rápido. Mas não ajusta o pré-treinamento.
# Assim que será desmonstrado agora....

#Copiando da sessão anterior...
parametros = {'n_neighbors': [3,5,7]}

GridSearchKNN = GridSearchCV(KNeighborsRegressor(), parametros, scoring='neg_root_mean_squared_error') # Da um nome para o GridSearchCV com KNN

modelo = Pipeline([
    ("padronização", StandardScaler()), # Faz a padronização utilizando o StandardScaler
    ("gsknn", GridSearchKNN) # E utilizasr o GridSearchKNN
])

#Copiando da sessão anterior (sem padronização) só que agora colocando com padronização
scores = cross_validate(modelo, X, y, scoring=make_scorer(mean_squared_error, squared=False))
print(scores['test_score'])
compad = np.mean(scores['test_score'])
print(f"Com padronização: {compad}") # Imprimindo com padronização

[56.82092904 63.05450375 60.15515846 54.92728369 59.33477082]
Com padronização: 58.858529150715086


In [12]:
# Continuando...
# Há duas formas de fazer:
# A segunda dela é simular que é necessário fazer algum ajuste pre-treino. No caso coloca-se o pipeline dentro do GridSearchCV. Porémn é mais lento

#Copiando da sessão anterior e ajustando...
pipeline = Pipeline([ #Antes o modelo era o pipeline, agora o modelo entra no GridSearchCV
    ("padronização", StandardScaler()),
    ("knn", KNeighborsRegressor())
])

parametros = {'knn__n_neighbors': [3,5,7]}

modelo = GridSearchCV(pipeline, parametros, scoring='neg_root_mean_squared_error')

scores = cross_validate(modelo, X, y, scoring=make_scorer(mean_squared_error, squared=False))
print(scores['test_score'])
compad = np.mean(scores['test_score'])
print(f"Com padronização: {compad}")

[56.82092904 63.05450375 60.15515846 54.92728369 59.33477082]
Com padronização: 58.858529150715086
